**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install sk-video

     |████████████████████████████████| 2.3MB 2.7MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

**Answer:**  
In the Agent class defined above, the function act, takes a certain action given its current state. The way it is coded, translates the $\epsilon$-greedy exploration when we take a random action with probability $\epsilon$ and take the action that maximizes the Q-function otherwise.  

$\epsilon$ controls how greedy is our exploration of the state space. It determines the tradeoff between exploration and exploitation. When $\epsilon$ is too large, we explore too much but do not exploit what we have already learned. When $\epsilon$ is too small, we do not do enough exploration to learn the best action.



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**Answer:**  
The array *board* is usde to store the reward of being in each cell over the $N\times N$ grid. If the agent is in a certain cell that we can access through the coordinates x and y, one reward is collected by him during an episode, this reward is reset to 0.

The array *position* defines the allowed states and the position of the agent, the borders (forbidden states) of the grid. At each step of the game, a value of -1 is assigned for the cells representing the borders , a value of 0 is assigned for all the other cells that can be visited, and a value of 1 is assigned for the current cell occupied by the agent.  
In conclusion, the arrays *position* and *board* are essential to describe the environment.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action) # randomly choose an action among possible actions with probability 1/n_actions
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/13.0. Average score (-2.0)
Win/lose count 8.0/12.0. Average score (-3.0)
Win/lose count 11.0/18.0. Average score (-4.333333333333333)
Win/lose count 10.5/21.0. Average score (-5.875)
Win/lose count 8.0/12.0. Average score (-5.5)
Win/lose count 16.0/19.0. Average score (-5.083333333333333)
Win/lose count 10.0/13.0. Average score (-4.785714285714286)
Win/lose count 10.5/10.0. Average score (-4.125)
Win/lose count 16.5/14.0. Average score (-3.388888888888889)
Win/lose count 10.5/8.0. Average score (-2.8)
Win/lose count 11.5/14.0. Average score (-2.772727272727273)
Win/lose count 10.5/9.0. Average score (-2.4166666666666665)
Win/lose count 5.0/15.0. Average score (-3.0)
Win/lose count 7.0/18.0. Average score (-3.5714285714285716)
Win/lose count 8.0/16.0. Average score (-3.8666666666666667)
Final score: -3.8666666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer:**  
We have : 
$$
\begin{align*} 
Q^\pi(s,a)& = \mathbb{E}_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=  \mathbb{E}_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          & = r(s,a) + \gamma \mathbb{E}_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
          & = r(s,a) + \mathbb{E}_{(s',a')\sim p(.|s,a)}\left(\gamma \mathbb{E}_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          & = \mathbb{E}_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma \mathbb{E}_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          & = \mathbb{E}_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a') ]
\end{align*}
$$
  

Since $\pi^*(s') = arg\max_{a'} Q^{*}(s',a')$, is a deterministic policy.

$$
\begin{align*}
Q^*(s,a)& = \mathbb{E}_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{*}(s',a') ]\\
        & = \mathbb{E}_{s'\sim p(.|s,a)\,and\,a'\sim \pi^*(.|s')}[r(s,a) + \gamma Q^{*}(s',a') ]\\
        & = \mathbb{E}_{s' \sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}Q^{*}(s',a') ]\\
\end{align*}
$$

Finally, we have shown that $Q^*(s,a)=\mathbb{E}_{s' \sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}Q^{*}(s',a') ]$, therefore, $Q^*$ is the solution of the equation $B(Q) = 0$ with $$B(Q) = \mathbb{E}_{s' \sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)]$$

Minimizing $\mathcal{L}(\theta)=\mathbb{E}_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will result in finding the optimal Q.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0) #free the memory when it exceeds max_size
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
from warnings import filterwarnings
filterwarnings('ignore')

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # predict the Q function by the current model and take the action which maximizes the Q function
        q = self.model.predict(s.reshape((1, 5, 5, self.n_state)))
        a = np.argmax(q)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # Saving current state
            input_states[i] = s_
            
            # Get current Q(s,.)
            target_q[i] = self.model.predict(s_[None])
            
            # Update Q(s,a)
            if game_over_:
                ######## FILL IN
                # There is no next state, we take only the reward
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                # We update target_q by computing the Temporal-Difference error 
                target_q[i,a_] = r_ + self.discount*np.max(self.model.predict(n_s_[None]),axis=1)
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        # Flattening the input frame
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        # First fully connected layer
        model.add(Dense(20, activation='relu'))
        # Final fully connected layer
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
size = 13
T=200
temperature=0.3
epochs_train=50
epochs_test=15

In [34]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.1260 | Win/lose count 6.0/7.0 (-1.0)
Epoch 001/050 | Loss 0.4354 | Win/lose count 3.5/6.0 (-2.5)
Epoch 002/050 | Loss 0.5512 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/050 | Loss 0.3130 | Win/lose count 1.5/4.0 (-2.5)
Epoch 004/050 | Loss 0.4777 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/050 | Loss 0.2317 | Win/lose count 3.5/8.0 (-4.5)
Epoch 006/050 | Loss 0.4351 | Win/lose count 7.5/9.0 (-1.5)
Epoch 007/050 | Loss 0.3572 | Win/lose count 4.0/6.0 (-2.0)
Epoch 008/050 | Loss 0.5151 | Win/lose count 3.0/4.0 (-1.0)
Epoch 009/050 | Loss 0.4981 | Win/lose count 3.0/6.0 (-3.0)
Epoch 010/050 | Loss 0.4338 | Win/lose count 10.5/14.0 (-3.5)
Epoch 011/050 | Loss 0.3544 | Win/lose count 6.5/5.0 (1.5)
Epoch 012/050 | Loss 0.4402 | Win/lose count 6.5/2.0 (4.5)
Epoch 013/050 | Loss 1.0888 | Win/lose count 4.0/3.0 (1.0)
Epoch 014/050 | Loss 1.3942 | Win/lose count 8.5/2.0 (6.5)
Epoch 015/050 | Loss 0.9279 | Win/lose count 2.5/1.0 (1.5)
Epoch 016/050 | Loss 0.6673 | Win/lose count 

In [40]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0200 | Win/lose count 4.0/2.0 (2.0)
Epoch 001/050 | Loss 0.0003 | Win/lose count 7.5/10.0 (-2.5)
Epoch 002/050 | Loss 0.0976 | Win/lose count 6.5/4.0 (2.5)
Epoch 003/050 | Loss 0.2121 | Win/lose count 3.5/4.0 (-0.5)
Epoch 004/050 | Loss 0.2517 | Win/lose count 6.5/14.0 (-7.5)
Epoch 005/050 | Loss 0.0809 | Win/lose count 4.5/5.0 (-0.5)
Epoch 006/050 | Loss 0.1599 | Win/lose count 4.5/1.0 (3.5)
Epoch 007/050 | Loss 0.1794 | Win/lose count 4.5/7.0 (-2.5)
Epoch 008/050 | Loss 0.2842 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/050 | Loss 0.2236 | Win/lose count 3.0/6.0 (-3.0)
Epoch 010/050 | Loss 0.1546 | Win/lose count 2.5/3.0 (-0.5)
Epoch 011/050 | Loss 0.6005 | Win/lose count 3.0/4.0 (-1.0)
Epoch 012/050 | Loss 0.2365 | Win/lose count 3.5/7.0 (-3.5)
Epoch 013/050 | Loss 0.3900 | Win/lose count 5.0/4.0 (1.0)
Epoch 014/050 | Loss 0.3984 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/050 | Loss 0.9254 | Win/lose count 6.5/7.0 (-0.5)
Epoch 016/050 | Loss 1.2056 | Win/lose count

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential()
        model.add(Conv2D(16, (3,3), input_shape=(5,5,self.n_state), activation="relu"))
        model.add(Conv2D(32, (3,3), activation="relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [26]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0001 | Win/lose count 1.5/3.0 (-1.5)
Epoch 001/050 | Loss 0.0000 | Win/lose count 6.0/12.0 (-6.0)
Epoch 002/050 | Loss 0.0002 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/050 | Loss 0.0002 | Win/lose count 2.5/6.0 (-3.5)
Epoch 004/050 | Loss 0.0008 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/050 | Loss 0.0001 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/050 | Loss 0.0046 | Win/lose count 4.5/3.0 (1.5)
Epoch 007/050 | Loss 0.0159 | Win/lose count 3.0/1.0 (2.0)
Epoch 008/050 | Loss 0.0165 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/050 | Loss 0.0001 | Win/lose count 3.0/1.0 (2.0)
Epoch 010/050 | Loss 0.0267 | Win/lose count 1.5/2.0 (-0.5)
Epoch 011/050 | Loss 0.0474 | Win/lose count 5.0/5.0 (0.0)
Epoch 012/050 | Loss 0.0742 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/050 | Loss 0.0516 | Win/lose count 2.0/3.0 (-1.0)
Epoch 014/050 | Loss 0.1939 | Win/lose count 8.0/4.0 (4.0)
Epoch 015/050 | Loss 0.2069 | Win/lose count 6.5/6.0 (0.5)
Epoch 016/050 | Loss 0.2800 | Win/lose count 7.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [36]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.5/2.0. Average score (1.5)
Win/lose count 4.5/0. Average score (3.0)
Win/lose count 10.5/1.0. Average score (5.166666666666667)
Win/lose count 7.5/2.0. Average score (5.25)
Win/lose count 3.0/2.0. Average score (4.4)
Win/lose count 5.0/2.0. Average score (4.166666666666667)
Win/lose count 2.0/0. Average score (3.857142857142857)
Win/lose count 6.5/3.0. Average score (3.8125)
Win/lose count 3.0/1.0. Average score (3.611111111111111)
Win/lose count 9.0/2.0. Average score (3.95)
Win/lose count 2.0/1.0. Average score (3.6818181818181817)
Win/lose count 3.5/0. Average score (3.6666666666666665)
Win/lose count 8.5/8.0. Average score (3.423076923076923)
Win/lose count 4.5/3.0. Average score (3.2857142857142856)
Win/lose count 3.0/2.0. Average score (3.1333333333333333)
Final score: 3.1333333333333333
Test of the FC
Win/lose count 3.0/3.0. Average score (0.0)
Win/lose count 6.0/2.0. Average score (2.0)
Win/lose count 7.5/10.0. Average score (0.5)
Win/lose count

In [28]:
HTML(display_videos('cnn_test10.mp4'))

In [29]:
HTML(display_videos('fc_test10.mp4'))

**Comments:**

* We observe that the agent doesn't explore the map very well and sometimes gets stuck in a position, when there is no positive reward within his visibility, especially for the agent with the fully connected network.

* When we increase the temperature, positive rewards cells get numerous, which helps the agent explore the map since more positive cells gets in his visibility, and therefore the final average score gets higher.

* The performance of CNN is better than that of Fully connected network (according to average score).


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    # Starting epsilon
    epsilon_start = agent.epsilon
    
    # Minimum value that epsilon could get
    epsilon_stop= 0.1

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Exponential decay for epsilon-greedy exploration
        agent.set_epsilon((epsilon_start-epsilon_stop)*np.exp(-e/8) + epsilon_stop)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f}) | Epsilon : {:.2f}"
              .format(e, epoch, loss, win, lose, win-lose, agent.epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0
        
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            

        self.malus_position[self.x, self.y] = 0.1 
        self.t = self.t + 1
        
        reward = self.board[self.x, self.y] - int(train)*self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0

        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)  
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2) 

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [54]:
# Parameters
size = 13
T=200
temperature=0.3
epochs_train=60
epochs_test=15

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')


Epoch 000/060 | Loss 0.0697 | Win/lose count 6.40/28.50 (-22.10) | Epsilon : 0.70
Epoch 001/060 | Loss 0.2462 | Win/lose count 9.60/27.70 (-18.10) | Epsilon : 0.63
Epoch 002/060 | Loss 0.2130 | Win/lose count 11.60/34.20 (-22.60) | Epsilon : 0.57
Epoch 003/060 | Loss 0.7030 | Win/lose count 7.20/26.30 (-19.10) | Epsilon : 0.51
Epoch 004/060 | Loss 0.2601 | Win/lose count 8.00/27.10 (-19.10) | Epsilon : 0.46
Epoch 005/060 | Loss 0.2091 | Win/lose count 12.00/33.10 (-21.10) | Epsilon : 0.42
Epoch 006/060 | Loss 0.2918 | Win/lose count 8.40/27.00 (-18.60) | Epsilon : 0.38
Epoch 007/060 | Loss 1.2761 | Win/lose count 4.40/21.00 (-16.60) | Epsilon : 0.35
Epoch 008/060 | Loss 0.5271 | Win/lose count 6.40/20.50 (-14.10) | Epsilon : 0.32
Epoch 009/060 | Loss 0.5653 | Win/lose count 8.80/25.90 (-17.10) | Epsilon : 0.29
Epoch 010/060 | Loss 1.0443 | Win/lose count 5.20/25.80 (-20.60) | Epsilon : 0.27
Epoch 011/060 | Loss 1.3893 | Win/lose count 5.20/27.80 (-22.60) | Epsilon : 0.25
Epoch 012/060 

FileNotFoundError: ignored

In [55]:
HTML(display_videos('cnn_train_explore10.mp4'))

In [56]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 15.5/3.0. Average score (12.5)
Win/lose count 9.5/3.0. Average score (9.5)
Win/lose count 12.5/7.0. Average score (8.166666666666666)
Win/lose count 7.0/4.0. Average score (6.875)
Win/lose count 3.0/0. Average score (6.1)
Win/lose count 8.5/3.0. Average score (6.0)
Win/lose count 12.5/3.0. Average score (6.5)
Win/lose count 9.5/0. Average score (6.875)
Win/lose count 7.0/5.0. Average score (6.333333333333333)
Win/lose count 8.0/3.0. Average score (6.2)
Win/lose count 9.5/3.0. Average score (6.2272727272727275)
Win/lose count 6.5/1.0. Average score (6.166666666666667)
Win/lose count 9.0/1.0. Average score (6.3076923076923075)
Win/lose count 5.5/1.0. Average score (6.178571428571429)
Win/lose count 11.0/1.0. Average score (6.433333333333334)
Final score: 6.433333333333334


**Comment:**

We can see that the final score is larger than the original DQN_CNN and the rat can explore almost the total map and visit almost all cheese cells. One may add more epochs for training on order to get a better performance which means reduce the loses.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
def GenerateWinningGames(agent, env, epochs, memory_size):
    """
    function to generate winning games by the expert DQN
    returns a memory which stores all (state, next_state, action, reward, game_over) generated by the expert DQN
    """
    # memory of winning games
    memory = Memory(memory_size)
    
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action in "test" mode (train = False)
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            m = [prev_state, state, action, reward, game_over]
            memory.remember(m)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Update stats
        score = score + win - lose
        
        if e % 10 == 0:
            print("Game {}: Win/lose count {}/{}. Average score ({})"
                  .format(e+1, win, lose, score/(1+e)))
    
    return memory

In [0]:
"""
  define the agent Mimic_DQN which takes the memory generated by the expert DQN as a parameter. 
  And during training, it gets random samples from this memory.
  Different from DQN, it does not update the memory by its own (state, action) experience as it just mimicks the DQN.
"""
class Mimic_DQN(Agent):
    def __init__(self, grid_size, memory, epsilon = 0.1, batch_size = 16, n_state=2):
        super(Mimic_DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = memory
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # predict the Q function by the current model and take the action which maximizes the Q function
        q = self.model.predict(s.reshape((1, 5, 5, self.n_state)))
        a = np.argmax(q)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
#         self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # randomly take a sample from memory
            m = self.memory.random_access()
            s = m[0].reshape((1, 5, 5, self.n_state))
            n_s = m[1].reshape((1, 5, 5, self.n_state))
            a = m[2]
            r = m[3]
            game_over = m[4]
            # store the state as a sample in the batch and predict the Q function by the current model
            input_states[i] = s
            target_q[i] = self.model.predict(s)
            
            # change target_q[i, a] to the estimated value in the two cases whether the game is over
            if game_over:
                ######## FILL IN
                target_q[i, a] = r
            else:
                ######## FILL IN
                target_q[i, a] = r + self.discount * np.max(self.model.predict(n_s))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class Mimic_DQN_CNN(Mimic_DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Mimic_DQN_CNN, self).__init__(*args,**kwargs)
        
        # design a CNN network with 2 conv layers of 16 and 32 filters and 1 fully connected layer
        
        ###### FILL IN
        model = keras.Sequential()
        model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
        model.add(keras.layers.Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
epochs_gen = 100
memory_size = T * epochs_gen
epsilon = 0.1

In [62]:
memory = GenerateWinningGames(agent, env, epochs_gen, memory_size)

Game 1: Win/lose count 4.0/0. Average score (4.0)
Game 11: Win/lose count 2.0/0. Average score (5.0)
Game 21: Win/lose count 4.0/0. Average score (4.761904761904762)
Game 31: Win/lose count 6.5/0. Average score (5.032258064516129)
Game 41: Win/lose count 2.0/0. Average score (4.719512195121951)
Game 51: Win/lose count 1.5/0. Average score (4.5588235294117645)
Game 61: Win/lose count 11.5/0. Average score (4.663934426229508)
Game 71: Win/lose count 10.0/0. Average score (4.753521126760563)
Game 81: Win/lose count 5.5/0. Average score (4.839506172839506)
Game 91: Win/lose count 6.0/0. Average score (4.9010989010989015)


In [65]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
mimic_agent = Mimic_DQN_CNN(size, lr=0.1, memory=memory, epsilon = epsilon, batch_size = 32,n_state=3)
train(mimic_agent, env, epochs_train, prefix='mimic_cnn_train_explore')
HTML(display_videos('mimic_cnn_train_explore10.mp4'))

Epoch 000/060 | Loss 0.0034 | Win/lose count 3.0/5.0 (-2.0)
Epoch 001/060 | Loss 0.0015 | Win/lose count 8.0/1.0 (7.0)
Epoch 002/060 | Loss 0.0005 | Win/lose count 3.0/7.0 (-4.0)
Epoch 003/060 | Loss 0.0009 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/060 | Loss 0.0004 | Win/lose count 7.0/6.0 (1.0)
Epoch 005/060 | Loss 0.0620 | Win/lose count 6.0/9.0 (-3.0)
Epoch 006/060 | Loss 0.0016 | Win/lose count 7.5/3.0 (4.5)
Epoch 007/060 | Loss nan | Win/lose count 2.5/8.0 (-5.5)
Epoch 008/060 | Loss nan | Win/lose count 2.0/5.0 (-3.0)
Epoch 009/060 | Loss nan | Win/lose count 1.5/1.0 (0.5)
Epoch 010/060 | Loss nan | Win/lose count 1.5/4.0 (-2.5)
Epoch 011/060 | Loss nan | Win/lose count 2.0/1.0 (1.0)
Epoch 012/060 | Loss nan | Win/lose count 0.5/2.0 (-1.5)
Epoch 013/060 | Loss nan | Win/lose count 3.5/1.0 (2.5)
Epoch 014/060 | Loss nan | Win/lose count 1.0/4.0 (-3.0)
Epoch 015/060 | Loss nan | Win/lose count 2.5/6.0 (-3.5)
Epoch 016/060 | Loss nan | Win/lose count 2.5/2.0 (0.5)
Epoch 017/060 | L

In [66]:
test(mimic_agent,env,epochs_test,prefix='mimic_cnn_test_explore')
HTML(display_videos('mimic_cnn_test_explore10.mp4'))

Win/lose count 3.0/1.0. Average score (2.0)
Win/lose count 4.0/3.0. Average score (1.5)
Win/lose count 2.5/5.0. Average score (0.16666666666666666)
Win/lose count 2.5/4.0. Average score (-0.25)
Win/lose count 0.5/2.0. Average score (-0.5)
Win/lose count 3.0/1.0. Average score (-0.08333333333333333)
Win/lose count 1.0/4.0. Average score (-0.5)
Win/lose count 2.5/3.0. Average score (-0.5)
Win/lose count 1.0/4.0. Average score (-0.7777777777777778)
Win/lose count 1.0/6.0. Average score (-1.2)
Win/lose count 1.5/8.0. Average score (-1.6818181818181819)
Win/lose count 1.5/5.0. Average score (-1.8333333333333333)
Win/lose count 1.5/2.0. Average score (-1.7307692307692308)
Win/lose count 3.0/3.0. Average score (-1.6071428571428572)
Win/lose count 2.0/6.0. Average score (-1.7666666666666666)
Final score: -1.7666666666666666


***